In [49]:
import tensorflow.keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape
from tensorflow.keras.applications import mobilenet
import numpy as np
from tensorflow.contrib import lite
import tensorflow as tf
from tensorflow.keras import backend as K

In [50]:
base_model = mobilenet.MobileNet(include_top=True, weights='imagenet', input_shape=(224,224,3))

for layer in base_model.layers:
    layer.trainable = False
    
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

In [51]:
outputs = base_model.get_layer('conv_preds').output
outputs = Flatten()(outputs)
outputs = Dense(2, activation='softmax')(outputs)

In [52]:
model = Model(inputs=base_model.input, outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=1e-3),
              metrics=['accuracy'])

In [53]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        'data/train',
        target_size=(224, 224),
        batch_size=15, 
        class_mode='categorical', interpolation='bilinear')

test_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = test_datagen.flow_from_directory(
        'data/validation',
        target_size=(224, 224),
        batch_size=18, 
        class_mode='categorical', interpolation='bilinear')

Found 30 images belonging to 2 classes.
Found 18 images belonging to 2 classes.


In [54]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=20,
        epochs=3,
        validation_data=validation_generator,
        validation_steps=5)

Epoch 1/3
20/20 [==============================] - 19s 927ms/step - loss: 1.3407 - acc: 0.6133 - val_loss: 1.4929 - val_acc: 0.5556
Epoch 2/3
20/20 [==============================] - 17s 846ms/step - loss: 0.0321 - acc: 1.0000 - val_loss: 1.2151 - val_acc: 0.6111
Epoch 3/3
20/20 [==============================] - 17s 867ms/step - loss: 0.0078 - acc: 1.0000 - val_loss: 1.1157 - val_acc: 0.6111


In [56]:
import os

for filename in os.listdir('data/validation/anamaria'):
    filename = os.path.join('data/validation/anamaria', filename)
    img = keras.preprocessing.image.load_img(filename, target_size=(224,224))
    img_array = keras.preprocessing.image.img_to_array(img) / 255
    predict = model.predict(img_array[None,...])
    print('É a Ana Maria? %.2f%%' % (predict[0][0] * 100))

É a Ana Maria? 99.95%
É a Ana Maria? 99.95%
É a Ana Maria? 4.30%
É a Ana Maria? 92.53%
É a Ana Maria? 25.10%
É a Ana Maria? 97.65%
É a Ana Maria? 92.95%
É a Ana Maria? 96.54%
É a Ana Maria? 99.62%


In [57]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph

In [58]:

frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])
tf.train.write_graph(frozen_graph, "/home/jeziel/Documents", "supla-anamaria.pb", as_text=False)

INFO:tensorflow:Froze 150 variables.
INFO:tensorflow:Converted 150 variables to const ops.


'/home/jeziel/Documents/supla-anamaria.pb'

In [60]:
input_arrays = ["input_1"]
output_arrays = ["dense/Softmax"]

converter = tf.contrib.lite.TocoConverter.from_frozen_graph(
  '/home/jeziel/Documents/supla-anamaria.pb', input_arrays, output_arrays)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

16904124